In [89]:
import torch_geometric.transforms
from torch_geometric.nn import knn_graph
import torch_geometric.data
import torch 
from torch_geometric.data import Data
import numpy as np
import pickle
from torch_geometric.data import DataLoader
import random
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import max_pool, graclus

from torch_geometric.nn import global_mean_pool



In [44]:
dataset = torch.load('defect_matrices/torch_test.pt')

In [63]:
print(f'First graph attributes:')
data = dataset[0]  # Get the first graph object.
#print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')

torch.manual_seed(1234)
train_dataset = dataset[:80]
test_dataset = dataset[80:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

First graph attributes:
Data(edge_attr=[72], edge_index=[2, 72], x=[9, 2], y=[4])
Number of nodes: 9
Number of edges: 72
Average node degree: 8.00
Number of training graphs: 80
Number of test graphs: 18


In [64]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

#for step, data in enumerate(train_loader):
    #print(f'Step {step + 1}:')
    #print('=======')
    #print(f'Number of graphs in the current batch: {data.num_graphs}')
    #print(data)
    

In [97]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(2, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        #self.conv3 = GCNConv(hidden_channels, hidden_channels)
        #self.lin = Linear(hidden_channels, 4)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        print(x.shape)
        x = self.conv1(x, edge_index)
        cluster_vec = graclus(edge_index)
        x = max_pool(cluster_vec,x)
        print(x.shape)
        x = self.conv2(x,edge_index)
        #x = x.relu()
        #x = self.conv2(x, edge_index)
        #x = x.relu()
        #x = self.conv3(x, edge_index)
        #print(x)
        
        # 2. Readout layer
        x = global_mean_pool(x, batch,4)  # [batch_size, hidden_channels]
        #print(x.shape)
        # 3. Apply a final classifier
        #x = F.dropout(x, p=0.5, training=self.training)
        #x = self.lin(x)
        #return x
        return F.log_softmax(x, dim=-1)

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(2, 64)
  (conv2): GCNConv(64, 64)
)


In [96]:
model = GCN(hidden_channels=64)
model = model.double()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        #print(out)
        #print(data.y.shape)
        loss = criterion(out,data.y.long())  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 2):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

torch.Size([10, 2])


AttributeError: 'Tensor' object has no attribute 'x'

In [132]:
aa = graclus(data.edge_index)
ei = data.edge_index

ei.reshape(-1)


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [119]:
cluster_vec = graclus(data.edge_index, weight=None, num_nodes=data.x.size(0))
#print(data.x)
#print(cluster_vec)
x = data.x
data.edge_attr = None
print(data)
x = max_pool(cluster_vec,data)

Data(edge_index=[2, 72], x=[9, 2], y=[4])


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [127]:
#cluster_vec[data.edge_index.view(-1)].view(2, -1)
data.edge_indexshape
#print(data.edge_index.view(-1))


torch.Size([2, 72])

In [84]:
target = torch.empty(6).type(torch.LongTensor)
print(target)

tensor([-9223372036854775808,                    0,                    0,
                           0,                    0,                    0])


In [230]:
torch.randint(0, 1, (4,))

tensor([0, 0, 0, 0])

In [227]:
print(a)

tensor([3])
